In [1]:
from openai import OpenAI
from langsmith.wrappers import wrap_openai
from openai import AzureOpenAI
from langsmith import traceable

from dotenv import load_dotenv

load_dotenv()

openai_client = wrap_openai(AzureOpenAI(
                azure_endpoint = "https://nitoropenai.openai.azure.com/",
                azure_deployment = "gpt-35-turbo-16k",
                api_version = "2023-07-01-preview",
                api_key = "78d406f975874c20ac2298c6d83e2f7d"
            ))

# This is the retriever we will use in RAG
# This is mocked out, but it could be anything we want
@traceable(run_type="retriever")
def retriever(query: str):
    results = ["Bonish is working at Nitor Infotech"]
    return results

# This is the end-to-end RAG chain.
# It does a retrieval step then calls OpenAI
@traceable
def rag(question):
    docs = retriever(question)
    system_message = """Answer the users question using only the provided information below:
    
    {docs}""".format(docs="\n".join(docs))
    
    return openai_client.chat.completions.create(
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": question},
        ],
        model="gpt-35-turbo-16k",
    )

In [2]:
rag("where did bonish work")

ChatCompletion(id='chatcmpl-AwopM4xrIrOJWgGnbnm86boiv6dCh', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Bonish worked at Nitor Infotech.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1738581768, model='gpt-35-turbo-16k', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=9, prompt_tokens=38, total_tokens=47, completion_tokens_details=None, prompt_tokens_details=None), prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, '